In [5]:
import sys

In [6]:
!pip install torch_geometric
!pip install grakel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 21.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path_to_modules = '/content/drive/MyDrive/lx/ALTEGRAD/final_project/baseline'
sys.path.append(path_to_modules)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
!pwd

/content


In [4]:
%cd /content/drive/MyDrive/lx/ALTEGRAD/final_project/baseline
!unzip data.zip -d /content
%cd /content


Streaming output truncated to the last 5000 lines.
  inflating: /content/__MACOSX/data/train/description/._graph_1592.txt  
  inflating: /content/data/train/description/graph_7685.txt  
  inflating: /content/__MACOSX/data/train/description/._graph_7685.txt  
  inflating: /content/data/train/description/graph_2925.txt  
  inflating: /content/__MACOSX/data/train/description/._graph_2925.txt  
  inflating: /content/data/train/description/graph_148.txt  
  inflating: /content/__MACOSX/data/train/description/._graph_148.txt  
  inflating: /content/data/train/description/graph_5092.txt  
  inflating: /content/__MACOSX/data/train/description/._graph_5092.txt  
  inflating: /content/data/train/description/graph_4398.txt  
  inflating: /content/__MACOSX/data/train/description/._graph_4398.txt  
  inflating: /content/data/train/description/graph_2931.txt  
  inflating: /content/__MACOSX/data/train/description/._graph_2931.txt  
  inflating: /content/data/train/description/graph_5086.txt  
  infl

In [9]:
from torch_geometric.loader import DataLoader

from autoencoder import VariationalAutoEncoder
from denoise_model import DenoiseNN, p_losses, sample
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error

from utils import linear_beta_schedule, sigmoid_beta_schedule, quadratic_beta_schedule, construct_nx_from_adj, preprocess_dataset
import torch.nn.functional as F
import torch
import numpy as np
import networkx as nx

In [10]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [8]:
#deafault args
lr = 1e-3
dropout = 0
batch_size = 256
epochs_autoencoder = 200
hidden_dim_encoder = 64
hidden_dim_decoder =  256
latent_dim = 32
n_max_nodes = 50
n_layers_encoder = 2
n_layers_decoder =3
spectral_emb_dim =10
epochs_denoise = 100
timesteps = 500
hidden_dim_denoise = 512
n_layers_denoise = 3
dim_condition = 128
n_condition = 7

In [20]:
# initialize VGAE model
EARLY_STOP_ROUNS = 10
MAX_NODES=50
SPECTR_EMB_DIM = 10
TRAIN_SIZE = 8000
VAL_SIZE = 1000

trainset = preprocess_dataset(f"train", MAX_NODES, SPECTR_EMB_DIM)
validset = preprocess_dataset("valid", MAX_NODES, SPECTR_EMB_DIM)
testset = preprocess_dataset("test", MAX_NODES, SPECTR_EMB_DIM)

100%|██████████| 8000/8000 [00:37<00:00, 214.34it/s]


Dataset ./data/dataset_train.pt saved


100%|██████████| 1000/1000 [00:04<00:00, 215.06it/s]


Dataset ./data/dataset_valid.pt saved
Dataset ./data/dataset_test.pt saved


In [11]:
def train_autoencoder(spectral_emb_dim,
                      hidden_dim_encoder,
                      hidden_dim_decoder,
                      latent_dim,
                      n_layers_encoder,
                      n_layers_decoder,
                      lr,
                      train_loader,
                      val_loader,
                      epochs_autoencoder=300,
                      ):
  autoencoder = VariationalAutoEncoder(spectral_emb_dim+1, hidden_dim_encoder, hidden_dim_decoder, latent_dim, n_layers_encoder, n_layers_decoder, n_max_nodes=MAX_NODES).to(DEVICE)
  optimizer = torch.optim.Adam(autoencoder.parameters(), lr=lr)
  scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.5, threshold=5e-3, min_lr=5e-6)
  early_stopping_counts = 0

  best_val_loss = np.inf
  for epoch in range(1, epochs_autoencoder + 1):
      autoencoder.train()
      train_loss_all = 0
      train_count = 0
      train_loss_all_recon = 0
      train_loss_all_kld = 0
      cnt_train=0

      for data in train_loader:
          data = data.to(DEVICE)
          optimizer.zero_grad()
          loss, recon, kld  = autoencoder.loss_function(data)
          train_loss_all_recon += recon.item()
          train_loss_all_kld += kld.item()
          cnt_train+=1
          loss.backward()
          train_loss_all += loss.item()
          train_count += torch.max(data.batch)+1
          optimizer.step()

      autoencoder.eval()
      val_loss_all = 0
      val_count = 0
      cnt_val = 0
      val_loss_all_recon = 0
      val_loss_all_kld = 0

      for data in val_loader:
          data = data.to(DEVICE)
          loss, recon, kld  = autoencoder.loss_function(data)
          val_loss_all_recon += recon.item()
          val_loss_all_kld += kld.item()
          val_loss_all += loss.item()
          cnt_val+=1
          val_count += torch.max(data.batch)+1

      # if epoch % 1 == 0:
      #     print('Epoch: {:04d}, Train Loss: {:.5f}, Train Reconstruction Loss: {:.2f}, Train KLD Loss: {:.2f}, Val Loss: {:.5f}, Val Reconstruction Loss: {:.2f}, Val KLD Loss: {:.2f}'.format(epoch, train_loss_all/TRAIN_SIZE, train_loss_all_recon/TRAIN_SIZE, train_loss_all_kld/TRAIN_SIZE, val_loss_all/VAL_SIZE, val_loss_all_recon/VAL_SIZE, val_loss_all_kld/VAL_SIZE))

      scheduler.step(val_loss_all)
      early_stopping_counts += 1

      if best_val_loss >= val_loss_all:
          best_val_loss = val_loss_all
          torch.save({
              'state_dict': autoencoder.state_dict(),
              'optimizer' : optimizer.state_dict(),
          }, 'autoencoder.pth.tar')
          early_stopping_counts = 0


      if early_stopping_counts == EARLY_STOP_ROUNS:
        break

  checkpoint = torch.load('autoencoder.pth.tar')
  autoencoder.load_state_dict(checkpoint['state_dict'])
  return autoencoder.eval()

In [12]:
def train_denoiser(
    n_layers_denoise,
    timesteps,
    n_condition,
    dim_condition,
    latent_dim,
    hidden_dim_denoise,
    autoencoder,
    epochs_denoise,
    lr,
    beta_fn = linear_beta_schedule,
):

  betas = beta_fn(timesteps=timesteps)
  alphas = 1. - betas
  alphas_cumprod = torch.cumprod(alphas, axis=0)
  alphas_cumprod_prev = F.pad(alphas_cumprod[:-1], (1, 0), value=1.0)
  sqrt_recip_alphas = torch.sqrt(1.0 / alphas)
  sqrt_alphas_cumprod = torch.sqrt(alphas_cumprod)
  sqrt_one_minus_alphas_cumprod = torch.sqrt(1. - alphas_cumprod)
  posterior_variance = betas * (1. - alphas_cumprod_prev) / (1. - alphas_cumprod)

  denoise_model = DenoiseNN(input_dim=latent_dim, hidden_dim=hidden_dim_denoise, n_layers=n_layers_denoise, n_cond=n_condition, d_cond=dim_condition).to(DEVICE)
  optimizer = torch.optim.Adam(denoise_model.parameters(), lr=lr)
  scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.5, threshold=5e-3, min_lr=5e-6)

  best_val_loss = np.inf
  early_stopping_counts = 0

  for epoch in range(1, epochs_denoise+1):
      denoise_model.train()
      train_loss_all = 0
      train_count = 0
      for data in train_loader:
          data = data.to(DEVICE)
          optimizer.zero_grad()
          x_g = autoencoder.encode(data)
          t = torch.randint(0, timesteps, (x_g.size(0),), device=DEVICE).long()
          loss = p_losses(denoise_model, x_g, t, data.stats, sqrt_alphas_cumprod, sqrt_one_minus_alphas_cumprod, loss_type="huber")
          loss.backward()
          train_loss_all += x_g.size(0) * loss.item()
          train_count += x_g.size(0)
          optimizer.step()

      denoise_model.eval()
      val_loss_all = 0
      val_count = 0
      for data in val_loader:
          data = data.to(DEVICE)
          x_g = autoencoder.encode(data)
          t = torch.randint(0, timesteps, (x_g.size(0),), device=DEVICE).long()
          loss = p_losses(denoise_model, x_g, t, data.stats, sqrt_alphas_cumprod, sqrt_one_minus_alphas_cumprod, loss_type="huber")
          val_loss_all += x_g.size(0) * loss.item()
          val_count += x_g.size(0)

      scheduler.step(val_loss_all)
      early_stopping_counts += 1
      # if epoch % 5 == 0:
      #   print('Epoch: {:04d}, Train Loss: {:.5f}, Val Loss: {:.5f}'.format(epoch, train_loss_all/TRAIN_SIZE, val_loss_all/VAL_SIZE))

      if best_val_loss >= val_loss_all:
          best_val_loss = val_loss_all
          torch.save({
              'state_dict': denoise_model.state_dict(),
              'optimizer' : optimizer.state_dict(),
          }, 'denoise_model.pth.tar')
          early_stopping_counts = 0

      if early_stopping_counts == EARLY_STOP_ROUNS:
        # print('early_stopping', best_val_loss, scheduler.get_last_lr())
        break

  checkpoint = torch.load('denoise_model.pth.tar')
  denoise_model.load_state_dict(checkpoint['state_dict'])
  denoise_model.eval()
  return denoise_model, betas

In [13]:
# apply the pipeline and eval
def eval(loader, autoencoder, denoiser, betas):
  targets = []
  preds = []

  with torch.no_grad():
    for data in loader:
      data = data.to(DEVICE)
      stat = data.stats
      targets.append(stat.cpu().numpy())
      bs = stat.size(0)
      samples = sample(denoiser, data.stats, latent_dim=latent_dim, timesteps=timesteps, betas=betas, batch_size=bs)
      x_sample = samples[-1]
      adj = autoencoder.decode_mu(x_sample)
      for i in range(bs):
        Gs_generated = construct_nx_from_adj(adj[i,:,:].detach().cpu().numpy())
        preds.append(get_features(Gs_generated))

  preds = np.array(preds)
  targets = np.concatenate(targets)

  scaler = StandardScaler()
  y_test_scaled = scaler.fit_transform(targets)
  y_pred_scaled = scaler.transform(preds)
  mae = mean_absolute_error(y_test_scaled, y_pred_scaled)
  return mae

In [14]:
def get_features(G):
    """ returns a list of features from G.
    We consider by default the features ['nodes', 'edges',
    'degree', 'triangles', 'clust_coef', 'max_k_core', 'communities']"""
    nodes = G.number_of_nodes()
    edges = G.number_of_edges()
    A = nx.adjacency_matrix(G)
    degree = A.sum(axis=1).mean()
    triangles = len(nx.triangles(G))
    clust_coef = nx.average_clustering(G)
    core_numbers = nx.core_number(G)
    max_k_core = max(core_numbers.values()) if core_numbers else 0
    #the number of communities is computed with an algorithm. I am choosing the one that is imported in utils.py
    #note that the algorithm yields a number of community with randomness, hence the averaging over 10 attemps and the float
    communities = np.array([len(nx.community.louvain_communities(G)) for _ in range(10)]).mean()

    return [nodes, edges, degree, triangles, clust_coef, max_k_core, communities]

In [15]:
from itertools import product


timesteps_list = [500, 1_000]
n_layers_denoise_list = [2, 3, 4]
dim_condition_list = [64, 128, 256]
latent_dim_list = [16, 32, 64]
hidden_dim_denoise_list = [256, 512, 768]
lr_list = np.linspace(1e-4, 1e-2, 3)
hidden_dim_encoder_list = [32, 64, 128]
hidden_dim_decoder_list = [128, 256, 512]
n_layers_encoder_list = [2, 3, 4]
n_layers_decoder_list = [2, 3, 4]
batch_size_list = [128, 256, 512]

params = list(product(
        timesteps_list,
        n_layers_denoise_list,
        dim_condition_list,
        latent_dim_list,
        hidden_dim_denoise_list,
        lr_list,
        batch_size_list,
        hidden_dim_encoder_list,
        hidden_dim_decoder_list,
        n_layers_encoder_list,
        n_layers_decoder_list
        ))

n_params = len(params)

In [16]:
from tqdm import tqdm

In [17]:
torch.set_warn_always(False)
import warnings
warnings.filterwarnings('ignore')

In [40]:
res = []
n_samples = 100
params_ind = np.random.choice(n_params, size=n_samples, replace=False)
for i in tqdm(params_ind):
  timesteps, n_layers_denoise, dim_condition, latent_dim, hidden_dim_denoise, lr, batch_size, hidden_dim_encoder, hidden_dim_decoder, n_layers_encoder, n_layers_decoder = params[i]

  train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
  val_loader = DataLoader(validset, batch_size=batch_size, shuffle=False)

  encoder = train_autoencoder(SPECTR_EMB_DIM,
                            hidden_dim_encoder,
                            hidden_dim_decoder,
                            latent_dim,
                            n_layers_encoder,
                            n_layers_decoder,
                            lr,
                            train_loader,
                            val_loader)
  denoiser, betas = train_denoiser(
      n_layers_denoise,
      timesteps,
      7,
      dim_condition,
      latent_dim,
      hidden_dim_denoise,
      autoencoder=encoder,
      epochs_denoise=300,
      lr=lr,
      beta_fn = linear_beta_schedule,
  )
  mae = eval(val_loader, encoder, denoiser, betas)
  res.append(list(params[i]) + [mae])

res

100%|██████████| 100/100 [5:07:27<00:00, 184.48s/it]


[[500, 2, 64, 32, 768, 0.01, 128, 32, 128, 2, 2, 0.9080552465139927],
 [500, 3, 128, 64, 256, 0.01, 256, 64, 128, 2, 4, 0.892177969305815],
 [500, 3, 64, 32, 256, 0.0001, 512, 128, 512, 2, 2, 0.9803250351439895],
 [500, 4, 128, 32, 768, 0.01, 128, 64, 256, 3, 4, 0.8907962700451643],
 [500, 3, 64, 64, 256, 0.0001, 128, 32, 256, 4, 3, 0.9644586846064299],
 [500,
  3,
  64,
  32,
  256,
  0.005050000000000001,
  512,
  64,
  256,
  4,
  2,
  0.9103948024451202],
 [1000, 3, 256, 16, 768, 0.0001, 128, 128, 128, 4, 4, 0.9029637406835073],
 [1000,
  3,
  256,
  64,
  256,
  0.005050000000000001,
  256,
  128,
  512,
  2,
  3,
  0.8613745319727716],
 [500,
  3,
  256,
  32,
  768,
  0.005050000000000001,
  128,
  32,
  128,
  3,
  2,
  0.8365006344796347],
 [500, 3, 128, 32, 768, 0.01, 128, 128, 512, 4, 4, 0.8908656048582415],
 [500, 4, 64, 16, 512, 0.01, 256, 64, 512, 3, 2, 0.8338891930969005],
 [500, 3, 256, 16, 768, 0.01, 512, 32, 256, 4, 2, 0.8234342564373545],
 [500, 2, 128, 32, 512, 0.00

In [43]:
sorted(res, key=lambda x: -x[-1])

[[500, 4, 128, 32, 512, 0.0001, 256, 32, 128, 4, 2, 0.9948157718166081],
 [500, 3, 64, 32, 256, 0.0001, 256, 128, 128, 4, 2, 0.9915393590316877],
 [1000, 4, 64, 16, 512, 0.0001, 512, 128, 128, 2, 2, 0.9900326489839092],
 [500, 3, 64, 32, 256, 0.0001, 512, 128, 512, 2, 2, 0.9803250351439895],
 [500, 3, 256, 64, 512, 0.0001, 128, 128, 256, 2, 3, 0.979812859303904],
 [1000, 2, 128, 16, 256, 0.0001, 512, 128, 128, 4, 3, 0.9775938493081983],
 [1000, 4, 64, 64, 512, 0.0001, 512, 128, 512, 3, 2, 0.975045494688413],
 [1000, 2, 256, 32, 768, 0.0001, 256, 32, 128, 4, 2, 0.9708119955502204],
 [500, 2, 256, 64, 256, 0.0001, 512, 32, 256, 3, 2, 0.9700439025257079],
 [1000, 3, 128, 64, 768, 0.0001, 128, 64, 512, 2, 4, 0.9691427359740413],
 [1000, 4, 256, 64, 512, 0.0001, 256, 32, 256, 2, 3, 0.9687022909460037],
 [1000, 4, 128, 64, 256, 0.0001, 128, 128, 512, 4, 2, 0.9670506219245214],
 [500, 3, 64, 64, 256, 0.0001, 128, 32, 256, 4, 3, 0.9644586846064299],
 [500, 3, 64, 64, 256, 0.0001, 256, 64, 256,

In [18]:
from itertools import product


timesteps_list = [500]
n_layers_denoise_list = [4]
dim_condition_list = [128]
latent_dim_list = [16, 32, 64]
hidden_dim_denoise_list = [256, 512]
lr_list = [1e-2]
hidden_dim_encoder_list = [32, 64, 128]
hidden_dim_decoder_list = [128, 256, 512]
n_layers_encoder_list = [3]
n_layers_decoder_list = [2, 3]
batch_size_list = [128, 256]

params = list(product(
        timesteps_list,
        n_layers_denoise_list,
        dim_condition_list,
        latent_dim_list,
        hidden_dim_denoise_list,
        lr_list,
        batch_size_list,
        hidden_dim_encoder_list,
        hidden_dim_decoder_list,
        n_layers_encoder_list,
        n_layers_decoder_list
        ))

n_params = len(params)
n_params

216

In [21]:
res = []
n_samples = 20
params_ind = np.random.choice(n_params, size=n_samples, replace=False)
for i in tqdm(params_ind):
  timesteps, n_layers_denoise, dim_condition, latent_dim, hidden_dim_denoise, lr, batch_size, hidden_dim_encoder, hidden_dim_decoder, n_layers_encoder, n_layers_decoder = params[i]

  train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
  val_loader = DataLoader(validset, batch_size=batch_size, shuffle=False)

  encoder = train_autoencoder(SPECTR_EMB_DIM,
                            hidden_dim_encoder,
                            hidden_dim_decoder,
                            latent_dim,
                            n_layers_encoder,
                            n_layers_decoder,
                            lr,
                            train_loader,
                            val_loader)
  denoiser, betas = train_denoiser(
      n_layers_denoise,
      timesteps,
      7,
      dim_condition,
      latent_dim,
      hidden_dim_denoise,
      autoencoder=encoder,
      epochs_denoise=300,
      lr=lr,
      beta_fn = linear_beta_schedule,
  )
  mae = eval(val_loader, encoder, denoiser, betas)
  res.append(list(params[i]) + [mae])

sorted(res, key=lambda x: x[-1])

100%|██████████| 20/20 [33:24<00:00, 100.21s/it]


[[500, 4, 128, 16, 512, 0.01, 128, 32, 256, 3, 2, 0.8107342907175423],
 [500, 4, 128, 16, 512, 0.01, 128, 128, 512, 3, 2, 0.8269975750067038],
 [500, 4, 128, 16, 512, 0.01, 128, 64, 256, 3, 2, 0.8281858148066945],
 [500, 4, 128, 64, 512, 0.01, 256, 32, 256, 3, 3, 0.832456698638496],
 [500, 4, 128, 16, 256, 0.01, 128, 64, 256, 3, 3, 0.8331138733376869],
 [500, 4, 128, 16, 512, 0.01, 128, 32, 256, 3, 3, 0.8341143840981972],
 [500, 4, 128, 64, 512, 0.01, 256, 32, 128, 3, 3, 0.8350379966594714],
 [500, 4, 128, 64, 512, 0.01, 256, 32, 512, 3, 3, 0.8381623250382007],
 [500, 4, 128, 64, 512, 0.01, 128, 64, 128, 3, 3, 0.8414237712640128],
 [500, 4, 128, 32, 256, 0.01, 256, 128, 256, 3, 2, 0.8473898927453992],
 [500, 4, 128, 32, 256, 0.01, 256, 64, 256, 3, 2, 0.8481793747178522],
 [500, 4, 128, 16, 512, 0.01, 256, 32, 256, 3, 3, 0.8554248654237634],
 [500, 4, 128, 16, 256, 0.01, 256, 64, 512, 3, 3, 0.8646059331765297],
 [500, 4, 128, 64, 512, 0.01, 128, 64, 512, 3, 3, 0.8670392996941557],
 [500

In [26]:
timesteps, n_layers_denoise, dim_condition, latent_dim, hidden_dim_denoise, lr, batch_size, hidden_dim_encoder, hidden_dim_decoder, n_layers_encoder, n_layers_decoder = (500, 3, 128, 16, 512, 0.01, 128, 32, 128, 3, 2)

train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(validset, batch_size=batch_size, shuffle=False)

encoder = train_autoencoder(SPECTR_EMB_DIM,
                          hidden_dim_encoder,
                          hidden_dim_decoder,
                          latent_dim,
                          n_layers_encoder,
                          n_layers_decoder,
                          lr,
                          train_loader,
                          val_loader)
denoiser, betas = train_denoiser(
    n_layers_denoise,
    timesteps,
    7,
    dim_condition,
    latent_dim,
    hidden_dim_denoise,
    autoencoder=encoder,
    epochs_denoise=300,
    lr=lr,
    beta_fn = linear_beta_schedule,
)
mae = eval(val_loader, encoder, denoiser, betas)
mae

0.8989875976218296

In [27]:
timesteps, n_layers_denoise, dim_condition, latent_dim, hidden_dim_denoise, lr, batch_size, hidden_dim_encoder, hidden_dim_decoder, n_layers_encoder, n_layers_decoder = (500, 3, 128, 16, 512, 0.01, 128, 32, 128, 3, 2)

train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(validset, batch_size=batch_size, shuffle=False)

encoder = train_autoencoder(SPECTR_EMB_DIM,
                          hidden_dim_encoder,
                          hidden_dim_decoder,
                          latent_dim,
                          n_layers_encoder,
                          n_layers_decoder,
                          lr,
                          train_loader,
                          val_loader)
denoiser, betas = train_denoiser(
    n_layers_denoise,
    timesteps,
    7,
    dim_condition,
    latent_dim,
    hidden_dim_denoise,
    autoencoder=encoder,
    epochs_denoise=300,
    lr=lr,
    beta_fn = linear_beta_schedule,
)
mae = eval(val_loader, encoder, denoiser, betas)
mae

0.8311467342426627

In [29]:
import csv
n_condition = 7

test_loader = DataLoader(testset, batch_size=batch_size, shuffle=False)
with open("output.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    # Write the header
    writer.writerow(["graph_id", "edge_list"])
    for k, data in enumerate(tqdm(test_loader, desc='Processing test set',)):
        data = data.to(DEVICE)

        stat = data.stats
        bs = stat.size(0)

        graph_ids = data.filename

        samples = sample(denoiser, data.stats, latent_dim=latent_dim, timesteps=timesteps, betas=betas, batch_size=bs)
        x_sample = samples[-1]
        adj = encoder.decode_mu(x_sample)
        stat_d = torch.reshape(stat, (-1, n_condition))


        for i in range(stat.size(0)):
            stat_x = stat_d[i]

            Gs_generated = construct_nx_from_adj(adj[i,:,:].detach().cpu().numpy())
            stat_x = stat_x.detach().cpu().numpy()

            # Define a graph ID
            graph_id = graph_ids[i]

            # Convert the edge list to a single string
            edge_list_text = ", ".join([f"({u}, {v})" for u, v in Gs_generated.edges()])
            # Write the graph ID and the full edge list as a single row
            writer.writerow([graph_id, edge_list_text])

Processing test set: 100%|██████████| 8/8 [00:06<00:00,  1.19it/s]
